<h3>Import Necessary Libraries </h3>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator 
import keras
from keras import preprocessing as pr
from keras import applications 
from keras._tf_keras.keras.applications import ResNet50
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Dense
from keras._tf_keras.keras.applications import ResNet50
from keras._tf_keras.keras.applications.resnet50 import preprocess_input

<h3>Build Model</h3>

In [3]:
num_classes = 23

image_resize = 224

batch_size_training = 100
batch_size_validation = 100
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
) 


train_generator = data_generator.flow_from_directory(
    'Skin Disease/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


valid_generator=data_generator.flow_from_directory('Skin Disease/test',
                                                   target_size=(image_resize,image_resize),
                                                   batch_size=batch_size_validation,
                                                   class_mode='categorical')

steps_per_epoch_training = len(train_generator)

steps_per_epoch_validation = len(valid_generator)
num_epochs = 2

Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [14]:
model = Sequential()
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_regularizer = regularizers.l2(0.01)
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(525,activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
fit_resnet = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
156/156 [==============================] - 825s 5s/step - loss: 3.0061 - accuracy: 0.0817 - val_loss: 2.9575 - val_accuracy: 0.1079
Epoch 2/2
156/156 [==============================] - 804s 5s/step - loss: 2.9723 - accuracy: 0.0975 - val_loss: 2.9781 - val_accuracy: 0.0850


In [7]:
import tensorflow as tf
from keras import load_model
import cv2
valid_generator=data_generator.flow_from_directory('Skin Disease/test',
                                                   target_size=(image_resize,image_resize),
                                                   batch_size=batch_size_validation,
                                                   class_mode='categorical')

model=load_model('my_model.h5')

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
real_label = []
predicted_class = []

for folder in os.listdir(valid_generator):
    folder_path = os.path.join(valid_generator, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        img = preprocess_input(np.array([img]))  # Add an extra dimension for batching

        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(real_label, predicted_class)

In [ ]:
import seaborn as sns
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()